# Projet Final Apache Spark

**Nom Etudiant : *DIAGO*

**Prenom Etudiant: *Babacar*

**Classe : *Master 1 Big Data Analytics*


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [104]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._
import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._
import org.apache.spark.sql.SparkSession


In [105]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    appName("Data Processing").
    getOrCreate

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4e58b081


In [106]:
spark

res74: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4e58b081


In [107]:
spark.sparkContext.getConf.getAll.foreach(println)

(spark.repl.class.outputDir,/tmp/tmpl7c92qc_)
(spark.rdd.compress,True)
(spark.serializer.objectStreamReset,100)
(spark.master,local[*])
(spark.submit.pyFiles,)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.driver.port,37535)
(spark.repl.class.uri,spark://192.168.1.153:37535/classes)
(spark.driver.host,192.168.1.153)
(spark.app.name,spylon-kernel)
(spark.app.id,local-1610233620173)
(spark.ui.showConsoleProgress,true)


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [108]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [109]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", StringType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,StringType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true),...


In [110]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [111]:
fireDF.cache()

res76: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [112]:
fireDF.count()

res77: Long = 175296


In [113]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |

In [114]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [115]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [116]:
// Reponse 1

//types d'appels distincts
val  types_appels_distincts = fireDF
      .select("CallType")
      .na.drop("all") 
println("Nombre d'appels distincts : "+types_appels_distincts1.count())


Distinct count: 30


types_appels_distincts: org.apache.spark.sql.DataFrame = [CallType: string]


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [ ]:
// Reponse 2

val types_appels_distincts1 = types_appels_distincts.distinct()
      types_appels_distincts1.show(10, false)

Intitializing Scala interpreter ...

### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [120]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
/*
Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes
*/
val Delay_Sup_Cinq=newFireDF.where($"ResponseDelayedinMins" > 5)
Delay_Sup_Cinq.select("CallNumber","ResponseDelayedinMins").show(10)

+----------+---------------------+
|CallNumber|ResponseDelayedinMins|
+----------+---------------------+
|  20110315|                 5.35|
|  20120147|                 6.25|
|  20130013|                  5.2|
|  20140067|                  5.6|
|  20140177|                 7.25|
|  20150056|            11.916667|
|  20150254|             5.116667|
|  20150265|             8.633333|
|  20150265|             95.28333|
|  20150380|                 5.45|
+----------+---------------------+
only showing top 10 rows



Delay_Sup_Cinq: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, UnitID: string ... 26 more fields]


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [121]:
var fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res85: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [125]:
//Reponse 4
val  types_appels_courants = fireTSDF
  .groupBy("CallType").count()
  .orderBy($"count".desc)
types_appels_courants.show(10)

+-----------------+------+
|         CallType| count|
+-----------------+------+
| Medical Incident|113794|
|   Structure Fire| 23319|
|           Alarms| 19406|
|Traffic Collision|  7013|
+-----------------+------+
only showing top 4 rows



typ_appels_plus_courants: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, count: bigint]


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [126]:
//Reponse 5-a
val fireTSDF1 = fireTSDF.withColumn("Zipcode_CallType", 
                    concat(col("Zipcode"),lit("_"),col("CallType")))

fireTSDF01: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 27 more fields]


### Question 5-b
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [129]:
//Reponse 5-b

val codes_postaux = fireTSDF
     .filter(fireTSDF("Zipcode") === "94102" ||fireTSDF("Zipcode") === "94103")

val q_San_Francisco = codes_postaux 
     .filter(codes_postaux("City") === "SF")
    

q_San_Francisco.select("City","Zipcode").show()




+----+-------+
|City|Zipcode|
+----+-------+
|  SF|  94102|
|  SF|  94102|
|  SF|  94103|
|  SF|  94103|
|  SF|  94103|
|  SF|  94103|
|  SF|  94102|
|  SF|  94102|
|  SF|  94102|
|  SF|  94102|
|  SF|  94103|
|  SF|  94102|
|  SF|  94103|
|  SF|  94102|
|  SF|  94103|
|  SF|  94102|
|  SF|  94102|
|  SF|  94102|
|  SF|  94103|
|  SF|  94103|
+----+-------+
only showing top 20 rows



filtre1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, UnitID: string ... 26 more fields]
quartiers_San_Francisco: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [130]:
//Reponse 6
fireTSDF.select("responseDelayedinMins")
        .describe().show()

+-------+---------------------+
|summary|responseDelayedinMins|
+-------+---------------------+
|  count|               175296|
|   mean|    3.892364154521585|
| stddev|    9.378286226254206|
|    min|          0.016666668|
|    max|              1844.55|
+-------+---------------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [131]:
//Reponse 7-a

val nb_annees_distinctes = fireTSDF.select(year(fireTSDF("IncidentDate")))
                                .distinct()
                                .na.drop("all") 
println("le nombre d'années distinctes: "+nb_annees_distinctes.count())


le nombre d'années distinctes: 19


nombre_années_distinctes: org.apache.spark.sql.DataFrame = [year(IncidentDate): int]


In [165]:
nb_annees_distinctes.show() 

+------------------+
|year(IncidentDate)|
+------------------+
|              2003|
|              2007|
|              2018|
|              2015|
|              2006|
|              2013|
|              2014|
|              2004|
|              2012|
|              2009|
|              2016|
|              2001|
|              2005|
|              2000|
|              2010|
|              2011|
|              2008|
|              2017|
|              2002|
+------------------+



### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [133]:
//Reponse 7-b
/*
Ecrire ici votre code
*/


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [134]:
//Reponse 8
/*
Ecrire ici votre code
*/


### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [21]:
//Reponse 9
/*
Ecrire ici votre code
*/

### Question 10
**Comment relire les données stockée en format Parquet?**

In [22]:
//Reponse 10
/*
Ecrire ici votre code
*/

## FIN